## NLP Embedding

In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

c:\ProgramData\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


> SBERT: all-MiniLM-L6-v2

In [2]:
# 최초 코드 실행 시 모델 다운로드에 시간 소요
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [4]:
df = pd.read_json('ply_meta.json')
df

,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt
0,117162,12월의 재즈,"크리스마스, 겨울",353,2016-12-26 12:33:33.000
1,151638,12월의 크리스마스를 yeah,발라드,29,2009-12-22 09:11:45.000
2,27984,1월의 최신 EDM 소식,일렉,84,2015-01-29 16:22:19.000
3,48636,2000년대 힙합을 부탁해,"힙합, 랩",54,2015-06-09 17:25:51.000
4,30079,2008 그랜드민트 페스티벌,락,12,2008-12-23 19:56:46.000
...,...,...,...,...,...
46753,63967,기분업 에너지필 사운드굿 해외 EDM 일렉 선곡 2019년9월,"기분전환, 운동, 에너지, 일렉트로니카, EDM, 신나는, 하우스, 드라이브, 기분...",10,2019-12-21 11:22:39.000
46754,2531,J pop처음 듣는 사람도 중독된 듯 빠져드는 j pop 리스트,"기분전환, 힐링, 휴식",344,2015-12-30 19:41:08.000
46755,120713,들으면 설레는 드라마 및 영화 삽입곡&OST,"OST, 센치, 드라마, 설렘",9,2017-12-06 23:27:08.000
46756,135687,남자고백노래,"사랑, 설렘",14,2016-06-23 13:24:27.000


In [ ]:
df['embedding'] = df.apply(lambda row: model.encode(row.ply_title), axis = 1)

In [8]:
df # embedding column이 추가된 것을 확인할 수 있음
df.head(10)

,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt,embedding
0,117162,12월의 재즈,"크리스마스, 겨울",353,2016-12-26 12:33:33.000,"[-0.0036148538, 0.12269346, -0.036742635, -0.0..."
1,151638,12월의 크리스마스를 yeah,발라드,29,2009-12-22 09:11:45.000,"[-0.011159741, 0.07935732, -0.026214544, -0.05..."
2,27984,1월의 최신 EDM 소식,일렉,84,2015-01-29 16:22:19.000,"[-0.011181115, -0.03056669, 0.046715852, 0.024..."
3,48636,2000년대 힙합을 부탁해,"힙합, 랩",54,2015-06-09 17:25:51.000,"[-0.08739715, 0.14408775, 0.027805904, -0.0116..."
4,30079,2008 그랜드민트 페스티벌,락,12,2008-12-23 19:56:46.000,"[-0.0038566438, 0.014470583, 0.041877, 0.04026..."
5,52572,2010 2017 빌보드 1위곡 모음,"빌보드, 팝송추천, 팝송, 팝송모음",117,2017-07-27 10:37:19.000,"[-0.0035611985, 0.07935353, 0.06585096, -0.021..."
6,59658,2013떡춤 신나는노래 강추,일렉,19,2013-06-01 18:26:12.000,"[-0.024426417, 0.047797676, 0.09576701, -0.022..."
7,132181,2015년 뜨거운 시작 – 국내 힙합 앨범 추천,"힙합, 랩",23,2015-04-21 11:25:28.000,"[-0.031370215, 0.13578545, 0.06833871, -0.0398..."
8,72976,2015년 Top 5 일렉트로닉 뮤직 공연,일렉,12,2015-12-18 17:20:24.000,"[-0.0047692717, -0.02408407, 0.07261661, -0.05..."
9,144084,2015년 상반기 주목할 국내 재즈 앨범들 1,재즈,72,2015-08-13 11:13:45.000,"[0.0049727964, 0.13383397, 0.0596337, -0.03368..."


In [ ]:
# embedding이 추가된 데이터 저장
# df.to_json('ply_meta_emd.json', force_ascii=False)

In [13]:
# 코사인 유사도 계산 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 가장 유사한 플레이리스트 추출
def return_answer(question):
    embedding = model.encode(question)
    df['cos_sim'] = df.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return df.loc[df['cos_sim'].idxmax()]['ply_title']

In [17]:
# 입력한 제목과 가장 유사한 플레이리스트
return_answer("드라이브 할 때 듣기 좋은 음악")

'드라이브 할때  비올때 듣기 좋은 음악'

In [18]:
# 코사인 유사도가 높은 순으로 플레이리스트 3개 추출
df.sort_values(by='cos_sim', ascending=False).head(3)

,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt,embedding,cos_sim
41417,89539,드라이브 할때 비올때 듣기 좋은 음악,"비올때, 밤, 새벽",21,2020-04-22 12:56:03.000,"[-0.0191132203, 0.0779379755, -0.0298780520000...",0.981440
35232,21267,드라이브 할때 듣기 좋은 음악,"기분전환, 여행, 산책",33,2015-10-08 17:53:40.000,"[-0.0201754533, 0.0783285722, -0.0298796706, -...",0.980774
4334,90153,교외로 드라이브 떠나며 듣고 싶은 음악,"여행, 산책, 드라이브",14,2015-10-29 15:47:07.000,"[-0.018250830500000002, 0.07580951600000001, -...",0.975050
